In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import talib
from utils import utils
import os
%matplotlib inline

In [2]:
df = pd.read_csv('../data/USDJPY.csv')
df.sample(5)

,Gmt time,Open,High,Low,Close,Volume
53815,07.12.2011 23:00:00.000,77.653,77.692,77.636,77.684,1348.0601
38598,02.07.2009 20:00:00.000,95.915,95.975,95.845,95.960,6817.7002
84482,08.11.2016 07:00:00.000,104.396,104.473,104.344,104.431,8676.3799
7062,18.06.2004 18:00:00.000,108.748,108.888,108.683,108.815,73370.7969
59213,17.10.2012 19:00:00.000,78.979,79.060,78.963,78.979,3411.3101


In [3]:
df = utils.rename_columns_and_format(df)
df = utils.get_all_indicators(df)

df.dropna(inplace=True)
df.head()

,date,open,high,low,close,volume,rsi,ema_12,ema_26,upper_bband,middle_bband,lower_bband,macd,macd_signal,macd_hist,k,d
33,2003-01-08 06:00:00,120.436,120.687,120.331,120.596,11794.4004,69.010347,120.132198,119.921418,120.788292,119.79655,118.804808,0.211636,0.067038,0.144598,45.716382,50.669477
34,2003-01-08 07:00:00,120.572,120.656,120.347,120.407,10637.4004,62.713588,120.174475,119.957387,120.857165,119.83185,118.806535,0.217813,0.097193,0.120620,52.899983,48.448860
35,2003-01-08 08:00:00,120.404,120.632,120.324,120.508,12336.2002,64.573838,120.225787,119.998173,120.936120,119.87120,118.806280,0.228226,0.123399,0.104827,56.862745,51.826370
36,2003-01-08 09:00:00,120.538,120.584,120.235,120.319,13210.7002,58.674600,120.240127,120.021938,120.981654,119.90550,118.829346,0.218708,0.142461,0.076247,36.861357,48.874695
37,2003-01-08 10:00:00,120.316,120.567,120.273,120.496,12268.0996,62.160985,120.279492,120.057054,121.043257,119.95620,118.869143,0.222877,0.158544,0.064333,44.736596,46.153566


## Funcion para obtener el modelo

## Prueba utilizando indicadores individuales como inputs

In [4]:
WINDOW = 30
P_TRAIN = 0.8
EPOCHS = 100
BATCH_SIZE = 32

### Hacemos un head 500 pq solo es para pruebas

In [5]:
df = df.head(500)

In [6]:
columns = ['ema_12', 'ema_26', 'upper_bband', 
         'middle_bband', 'lower_bband', 'rsi', 
         'macd', 'macd_signal', 'macd_hist', 'k', 'd']

target_column = df['close'].to_numpy()
for column in columns:
    df_min = df[column]
    x, y = utils.create_windowed_dataset(df_min, target_column, WINDOW)
    x_train, x_test, y_train, y_test = utils.train_test_split(P_TRAIN, x, y)
    
    _, x_input, y_input = x.shape
    model = utils.get_model(x_input, y_input)
    
    x_train = np.expand_dims(x_train, axis=3)
    x_test = np.expand_dims(x_test, axis=3)
    
    history = model.fit(x_train, 
                    y_train, 
                    validation_data=(x_test,y_test), 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS,
                    verbose=0)
    
    df_history = pd.DataFrame(history.history)
    df_history.to_csv(f'../metrics/{column}_performance.csv', index=False) 

NameError: name 'tf' is not defined

In [ ]:
df_metrics = pd.DataFrame(columns=['loss','val_loss', 'root_mean_squared_error','val_root_mean_squared_error'])

files = os.listdir('../metrics')
for file in files:
    input_name = file
    
    path = os.path.join('../metrics', file)
    df_metric = pd.read_csv(path)
    
    df_metrics = df_metrics.append(df_metric.iloc[-1])

files = [f.split('.csv')[0] for f in files]
df_metrics.index = files
df_metrics.sort_values(by=['val_root_mean_squared_error'])


In [ ]:
#grafico de barras con la performance de c/u

## Prueba utilizando todos los indicadores

In [ ]:
df_min = df[columns]

x, y = utils.create_windowed_dataset(df_min, df['close'].to_numpy(), WINDOW)
x_train, x_test, y_train, y_test = utils.train_test_split(P_TRAIN, x, y)

_, x_input, y_input = x.shape
model = utils.get_model(x_input, y_input)

x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

history = model.fit(x_train, 
                y_train, 
                validation_data=(x_test,y_test), 
                batch_size=BATCH_SIZE, 
                epochs=EPOCHS,
                verbose=0)

In [ ]:
x1 = model.history.history['loss']
x2 = model.history.history['val_loss']
title = "Pérdidas del modelo con set de entrenamiento y pruebas por época"
ylabel = 'Pérdidas'
xlabel = 'Épocas'
legend = ['Entrenamiento', 'Validación']
utils.plot(x1, x2, title, xlabel, ylabel, legend)

x1 = model.history.history['root_mean_squared_error']
x2 = model.history.history['val_root_mean_squared_error']
title = "Error cuadratico medio del modelo con set de entrenamiento y pruebas por época"
ylabel = 'RMSE'
xlabel = 'Épocas'
legend = ['Entrenamiento', 'Validación']
utils.plot(x1, x2, title, xlabel, ylabel, legend)

## Prueba utilizando los indicadores de tipo oscilador 

In [ ]:
columns = ['rsi', 'macd', 'macd_signal', 'macd_hist', 'k', 'd']

df_min = df[columns]

x, y = utils.create_windowed_dataset(df_min, df['close'].to_numpy(), WINDOW)
x_train, x_test, y_train, y_test = utils.train_test_split(P_TRAIN, x, y)

_, x_input, y_input = x.shape
model = utils.get_model(x_input, y_input)

x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

history = model.fit(x_train, 
                y_train, 
                validation_data=(x_test,y_test), 
                batch_size=BATCH_SIZE, 
                epochs=EPOCHS,
                verbose=0)

In [ ]:
x1 = model.history.history['loss']
x2 = model.history.history['val_loss']
title = "Pérdidas del modelo con set de entrenamiento y pruebas por época"
ylabel = 'Pérdidas'
xlabel = 'Épocas'
legend = ['Entrenamiento', 'Validación']
utils.plot(x1, x2, title, xlabel, ylabel, legend)

x1 = model.history.history['root_mean_squared_error']
x2 = model.history.history['val_root_mean_squared_error']
title = "Error cuadratico medio del modelo con set de entrenamiento y pruebas por época"
ylabel = 'RMSE'
xlabel = 'Épocas'
legend = ['Entrenamiento', 'Validación']
utils.plot(x1, x2, title, xlabel, ylabel, legend)

## Prueba utilizando solo los indicadores de tendencia

In [ ]:
columns = ['ema_12', 'ema_26', 'upper_bband', 
         'middle_bband', 'lower_bband']

df_min = df[columns]

x, y = utils.create_windowed_dataset(df_min, df['close'].to_numpy(), WINDOW)
x_train, x_test, y_train, y_test = utils.train_test_split(P_TRAIN, x, y)

_, x_input, y_input = x.shape
model = utils.get_model(x_input, y_input)

x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

history = model.fit(x_train, 
                y_train, 
                validation_data=(x_test,y_test), 
                batch_size=BATCH_SIZE, 
                epochs=EPOCHS,
                verbose=0)

In [ ]:
x1 = model.history.history['loss']
x2 = model.history.history['val_loss']
title = "Pérdidas del modelo con set de entrenamiento y pruebas por época"
ylabel = 'Pérdidas'
xlabel = 'Épocas'
legend = ['Entrenamiento', 'Validación']
utils.plot(x1, x2, title, xlabel, ylabel, legend)

x1 = model.history.history['root_mean_squared_error']
x2 = model.history.history['val_root_mean_squared_error']
title = "Error cuadratico medio del modelo con set de entrenamiento y pruebas por época"
ylabel = 'RMSE'
xlabel = 'Épocas'
legend = ['Entrenamiento', 'Validación']
utils.plot(x1, x2, title, xlabel, ylabel, legend)